In [47]:
import pandas as pd
import numpy as np

import os
import warnings
warnings.filterwarnings('ignore')

**December 2018**

Påhl Ruin hittade statistik på livslängden i olika regioner inom EU. Vi kom på att det vore kul att göra en artikel om hur under förväntan svenska regioners livslängd är jämfört andra regioner i Europa. Resultatet gick på trendsidan i nr 1 2019. På webben [hittar man artikeln här](https://www.dagenssamhalle.se/nyhet/svenskar-lever-inte-langre-langst-25656)! Källa är [Eurostat](http://appsso.eurostat.ec.europa.eu/nui/show.do?dataset=demo_r_mlifexp&lang=en).

In [48]:
df = pd.read_csv('data/eurostat.csv', encoding="ISO-8859-1")

In [49]:
info = pd.read_csv('data/info.csv', encoding="ISO-8859-1")

Data från Eurostat är ganska krångligt formatterad. Därför krävs en del tvätt. Till varje datakörning på eurostats hemsida kan man dra ut en infofil med vad varje regionförkortning står för. Men här tvättas all data löpandes och sedan har jag gjort en del dubbelkollar för att se att allt stämmer. Men först tvätt:

In [50]:
info['region'] = info.GEO.str.extract(" - (.*)")
info['kod'] = info.GEO.str.extract("(.*) - ")

In [51]:
info.loc[(~info['GEO'].str.contains('\d+'))&(info.kod.str.len()<3),'land'] = \
info.loc[(~info['GEO'].str.contains('\d+'))&(info.kod.str.len()<3),'region']

Tyskland måste döpas om:

In [52]:
info.loc[info['GEO'].str.contains('DE'),'land'] = 'Germany'

In [53]:
info.land = info.land.ffill()

Nu lite viktig ytterligare tvätt. Här skapas kolumner med regionkoderna etc.Håll t ex koll på Londons regionindelning, det är något vajsing med den:

In [54]:
info.loc[info['region'].str.contains('London')]

,GEO,region,kod,land
376,UKI - London,London,UKI,United Kingdom
377,UKI3 - Inner London - West,Inner London - West,UKI3 - Inner London,United Kingdom
378,UKI4 - Inner London - East,Inner London - East,UKI4 - Inner London,United Kingdom
379,UKI5 - Outer London - East and North East,Outer London - East and North East,UKI5 - Outer London,United Kingdom
380,UKI6 - Outer London - South,Outer London - South,UKI6 - Outer London,United Kingdom
381,UKI7 - Outer London - West and North West,Outer London - West and North West,UKI7 - Outer London,United Kingdom


In [55]:
info['reg_kod'] = info.kod.str.extract('(\w+\d+)')

info.loc[info['kod'].str.contains('-'),'kod'] = \
info.loc[info['kod'].str.contains('-'),'reg_kod']


info = info.drop('reg_kod',axis=1)

In [56]:
info.head()

,GEO,region,kod,land
0,BE - Belgium,Belgium,BE,Belgium
1,BE1 - Région de Bruxelles-Capitale / Brussels ...,Région de Bruxelles-Capitale / Brussels Hoofds...,BE1,Belgium
2,BE10 - Région de Bruxelles-Capitale / Brussels...,Région de Bruxelles-Capitale / Brussels Hoofds...,BE10,Belgium
3,BE2 - Vlaams Gewest,Vlaams Gewest,BE2,Belgium
4,BE21 - Prov. Antwerpen,Prov. Antwerpen,BE21,Belgium


In [57]:
info = info[['kod','land','region']]

In [58]:
df.columns = df.columns.str.lower().str.replace('geo','kod').str.replace(' ','_')

In [59]:
df = df.drop('flag_and_footnotes',axis=1)

In [60]:
df.head()

,time,kod,kod_label,sex,age,unit,value
0,2016,BE,Belgium,Total,Less than 1 year,Year,81.5
1,2016,BE,Belgium,Males,Less than 1 year,Year,79.0
2,2016,BE,Belgium,Females,Less than 1 year,Year,84.0
3,2016,BE1,Région de Bruxelles-Capitale / Brussels Hoofds...,Total,Less than 1 year,Year,81.3
4,2016,BE1,Région de Bruxelles-Capitale / Brussels Hoofds...,Males,Less than 1 year,Year,78.4


In [61]:
test = info.merge(df[['kod','time','sex','age','unit','value']],on='kod',how='outer')

Senaste året med jämförbar data är 2016:

In [62]:
test = test.loc[test['time']==2016]

Alla EU-28-länder som vi vill jämföra emellan:

In [63]:
eu_28 = [
    'Austria',
    'Italy',
    'Belgium',
    'Latvia',
    'Bulgaria',
    'Lithuania',
    'Croatia',
    'Luxembourg',
    'Cyprus',
    'Malta',
    'Czechia',
    'Netherlands',
    'Denmark',
    'Poland',
    'Estonia',
    'Portugal',
    'Finland',
    'Romania',
    'France',
    'Slovakia',
    'Germany',
    'Slovenia',
    'Greece',
    'Spain',
    'Hungary',
    'Sweden',
    'Ireland',
    'United Kingdom'
]

Denna följande kod gör följande:

1. Filtrerar fram alla rader som _inte_ innehåller några siffror i kolumnen "kod". Dvs de rader som är enbart landdata, ej regiondata
2. Filtrerar fram data för båda könen
3. Filtrerar fram endast länder inom EU28
4. Sorterar efter vilket land som har högst medellivslängd
5. Filtrerar fram enbart kolumnerna "land" och "value" för resultatet

In [64]:
länderna = test.loc[  (~test['kod'].str.contains('\d'))\
                    & (test['kod'].str.len() < 3)\
                    & (test['sex'] == 'Total')\
                    & (test['land'].isin(eu_28))]\
    .sort_values('value',ascending=False)\
    .reset_index(drop=True)[['land',
                             'value']]

Nu ranking av resultatet:

In [65]:
länderna['rank'] = länderna['value'].rank(method='min',ascending=False).astype('int')

In [66]:
os.makedirs('res')

In [67]:
länderna.to_excel('res/efter_länder.xlsx',index=False)

In [68]:
df = test.loc[test['land'].isin(eu_28)]

Nu behöver vi kontrollera respektive region:

In [69]:
nuts = pd.read_csv('data/nuts2.csv', encoding="ISO-8859-1")

In [70]:
nuts['region'] = nuts.GEO.str.extract(" - (.*)")
nuts['kod'] = nuts.GEO.str.extract("(.*) - ")

In [71]:
nuts['nuts2'] = 'ja'

In [72]:
nuts['reg_kod'] = nuts.kod.str.extract('(\w+\d+)')

nuts.loc[nuts['kod'].str.contains('-'),'kod'] = \
nuts.loc[nuts['kod'].str.contains('-'),'reg_kod']


nuts = nuts.drop('reg_kod',axis=1)

In [73]:
df.head()

,kod,land,region,time,sex,age,unit,value
0,BE,Belgium,Belgium,2016,Total,Less than 1 year,Year,81.5
1,BE,Belgium,Belgium,2016,Males,Less than 1 year,Year,79.0
2,BE,Belgium,Belgium,2016,Females,Less than 1 year,Year,84.0
3,BE1,Belgium,Région de Bruxelles-Capitale / Brussels Hoofds...,2016,Total,Less than 1 year,Year,81.3
4,BE1,Belgium,Région de Bruxelles-Capitale / Brussels Hoofds...,2016,Males,Less than 1 year,Year,78.4


Dubbelkolla alla NUTS2-regioner:

In [74]:
buggcheck = nuts[['kod','nuts2']].merge(df[['kod','region']],on='kod',how='left')

In [75]:
not_included = buggcheck.loc[buggcheck['region'].isnull(),'kod']

In [76]:
info.loc[info['kod'].isin(not_included.tolist())].head()

,kod,land,region
404,IS00,Iceland,Ísland
407,LI00,Liechtenstein,Liechtenstein
410,NO01,Norway,Oslo og Akershus
411,NO02,Norway,Hedmark og Oppland
412,NO03,Norway,Sør-Østlandet


In [77]:
df = df.merge(nuts[['kod','nuts2']],on='kod',how='left')

In [78]:
df = df.dropna()

In [79]:
results = df.sort_values('value',ascending=False)

In [80]:
results.value = pd.to_numeric(results.value.str.replace(':',''))

In [81]:
results = results.loc[results['sex']=='Total'].sort_values('value',ascending=False).reset_index(drop=True)

In [82]:
results = results.reset_index().rename(columns={'index':'rank'})

In [83]:
results['rank'] = results['rank']+1

In [84]:
eu_28 = [
    'Austria',
    'Italy',
    'Belgium',
    'Latvia',
    'Bulgaria',
    'Lithuania',
    'Croatia',
    'Luxembourg',
    'Cyprus',
    'Malta',
    'Czechia',
    'Netherlands',
    'Denmark',
    'Poland',
    'Estonia',
    'Portugal',
    'Finland',
    'Romania',
    'France',
    'Slovakia',
    'Germany',
    'Slovenia',
    'Greece',
    'Spain',
    'Hungary',
    'Sweden',
    'Ireland',
    'United Kingdom'
]

Översättningar av alla länder:

In [85]:
eu_map = {
    'Austria':'Österrike',
    'Italy':'Italien',
    'Belgium':'Belgien',
    'Latvia':'Lettland',
    'Bulgaria':'Bulgarien',
    'Lithuania':'Litauen',
    'Croatia':'Kroatien',
    'Luxembourg':'Luxenburg',
    'Cyprus':'Cypern',
    'Malta':'Malta',
    'Czechia':'Tjeckien',
    'Netherlands':'Nederländerna',
    'Denmark':'Danmark',
    'Poland':'Polen',
    'Estonia':'Estland',
    'Portugal':'Portugal',
    'Finland':'Finland',
    'Romania':'Rumänien',
    'France':'Frankrike',
    'Slovakia':'Slovakien',
    'Germany':'Tyskland',
    'Slovenia':'Slovenien',
    'Greece':'Grekland',
    'Spain':'Spanien',
    'Hungary':'Ungern',
    'Sweden':'Sverige',
    'Ireland':'Irland',
    'United Kingdom':'Storbritannien'}


In [86]:
results.land = results.land.map(eu_map)

In [87]:
results = results[['rank','land','region','value']].dropna()

In [88]:
results['rank'] = results['value'].rank(method='min',ascending=False).astype('int')

In [89]:
pd.concat([pd.concat([results.head(10),results.loc[results['land']=='Sverige']]),results.tail(10)]).to_excel('res/regioner.xlsx',index=False)

In [90]:
results.to_excel('res/alla_regioner.xlsx',index=False)

In [91]:
results.shape

(275, 4)